# Controle continu 1 - compter les mots

On se propose dans ce TP d'étudier la fréquence de mots et leur pertinence dans plusieurs documents. On cherchera par la suite (dans un autre TP) à évaluer la similarité entre documents. ce type d'opérations s'inscrit dans une displine plus générale :  le traitement naturel du langage (NLP). Les algorithmes exploitant la fréquence de mots font partie d'une famille appelée "bag of words" très utilisée par les acteurs de la data.

 ### Exercice preparatoire : lire un fichier.
La lecture de données peut s'effectuer à la volée sur Internet (csv, json, xml, etc.) ou en "parsant" les données depuis des pages web (crawling). On se contentera de lire les données dans des fichiers.

Lire la première fable de lafontaine : fichier `lafontaine01.txt` par une des méthode de lecture proposée.

Il existe plusieurs techniques pour lire un fichier texte:

- Lire ce fichier d'une seule traite (déconseillé pour de gros fichiers)
```
descripteur = open("fichier.txt", "r", encoding='utf8')
chaineTexte = descripteur.read()
```
- Charger ce fichier ligne à ligne (on récupère une liste de lignes)
```
descripteur = open("fichier.txt", "r", encoding='utf8')
listelignes = descripteur.readlines()
```

- Lecture ligne avec traitement ligne par ligne
```
with open("fichier.txt", 'r', encoding='utf8') as descripteur:
    lignes = descripteur.readlines()
    for ligne in lignes:
        print(ligne)
```

### Warning: Choisissez le fichier à traiter dans la celulle concernant l'exercice 3,notamment en fournissant son path à la fonction cleaning_map 



<style>
p {
  color: #26b72b;
}
</style>

In [11]:
def read_file(path):
    descripteur = open(path,"r",encoding='utf8')
    
    ## Ici pour favoriser le traitement de données on mettra tout le texte en minuscule
    ## Cela sera beaucoup plus simple pour compter les mots et utiliser les fonctions intermédiaires
    raw_text_to_lower = map(lambda s : s.lower().strip(),descripteur.readlines())
    
    return raw_text_to_lower

with open("stopwords_fr.txt", "r", encoding='utf8') as f:
    ## en transformant stop_words en set, l'accession de données se fait beaucoup plus rapidement qu'avec une liste
    stop_words = set(f.read().split())

### 1. Preprocessing des données  - fonction cleaning_map(texte) 
Dans une première étape on cherchera à nettoyer les données par une fonction ``cleaning(texte)` qui prend en entrée un texte et renvoie la liste des mots de ce texte (map) ainsi nettoyés :
1. Suppprimer la ponctuation
2. Remplacer les tirets des mots liés par un espace (ex. celui-ci--> celui ci). Ainsi nous avons bien deux mots
3. Remplacer les mots contractés (taille 1 caractère) soit en les supprimant soit en les remplacant par leur équivalent non contractés  (exemple l --> le ou la au choix). on repère les mots contractés en cherchant les mots de une à deux lettres. ci-dessous le dictionnaire des mots contractés (`'d':'de', 's':'se', 'l':'le', 'j':'je', 'n':'ne', 'c':'ce'`)
4. Supprimer les stop-words (articles, pronom, etc.) seront supprimés. Une liste non exhaustive vous est fournie dans le fichier _*stopwords.txt*_

- Renvoyer cette liste de mots restante (ce que l'on appelle un map)

In [3]:
dico = {'d' : 'de',
        's' : 'se',
        'l' : 'le',
        'j' : 'je',
        'n' : 'ne',
        'c' : 'ce'}


    
def remove_punctuation_word(word):
    characters = list(word) # "word" -> "["w","o","r","d"]"
    
    ## on remplace toute la ponctuation par " " (pour gérer le tiret) en gardant les caractères alphanumériques
    ## Puis on retire les espaces inutiles au début et à la fin
    return "".join(map(lambda c : c if c.isalnum() else " ",characters)).strip()

def remove_punctuation(text):
    
    t = " ".join(text).split()
    return " ".join([remove_punctuation_word(word) for word in t])


def stemming(word):
    ## Si le mot est de longueur une et est dans le dico alors on retourne sa valeur associée
    if len(word) == 1 and word in dico:
        return dico[word]
    ## sinon on retourne le même mot
    else:
        return word
    
def replace_contracted_words(text):
    return map(stemming,text.split())
  
def remove_stop_words(text):
    
    return filter(lambda word : word not in stop_words and len(word) > 2  and not word.isspace(),text)

def cleaning_map(path):
    ## on lit le fichier voulu
    raw_text = read_file(path) 
    
    ## on enlève toute la ponctuation
    punctuation_removed = remove_punctuation(raw_text) 
    
    ## on convertit les mots à une lettre grâce au dico
    contracted_words_replaced = replace_contracted_words(punctuation_removed) 
    
    ## on enleve tous les stop_words du fichier, tous les mots de longueur < 2 et tous les espaces superflus
    stop_words_removed = remove_stop_words(contracted_words_replaced)
    
    
    return list(stop_words_removed)


### 2. Réduction des données - fonction reduce (word_count)
La liste de mots produite contient plusieurs fois les mêmes mots. Dans cette partie nous allons produire un dictionnaire contenant chaque mot (clé) et son occurrence (valeur) :
- Ecrire cette fonction `reduce(wordList)` qui 'reduit' notre liste de mots comme indiqué. Elle renvoie un dictionnaire:
```
toto 2
tutu 1
tonton 4
etc...
```

Cette façon de procéder *_map_* puis *_réduce_* a été inventée par Google pour réduire le texte concernant chaque compte utilisteur pour ne conserver que l'information pertinente. 
- La fonction `map()` fait l'inventaire des mots utiles
- La fonction `reduce()` réduit selon une règle (ici on compte les mots)
- L'ensemble des mots produits se nomme le corpus



In [2]:
def reduce(cleaned_words):
    ## les mots sont déjà tous en minuscule donc on peut directement effectuer le décompte de façon case-insensitive
    word_count = {}
    for word in cleaned_words:
        word_count[word] = word_count.get(word, 0) + 1
    return word_count

### 3. Affichage la distribution des mots 
Construire un graphique qui met en évidence la distribution des mots (en ordonnée la fréquence du mot et en abscisse les mots du corpus. 

On utilisera la librairie graphique Seaborn 
- https://seaborn.pydata.org/generated/seaborn.barplot.html

```
# import de la bibliothèque
import seaborn as sns 

#modifier la taille de la figure pour l'adapter
sns.set(rc={'figure.figsize':(20,10)})

# afficher un histogramme. les données doivent être dans une liste
# transformez donc vos clés et valeurs du dictionnaire en liste
# idéalement ordonnez vos clés
sns.barplot(data=None,  x=liste_des_x, y=liste_des_y)
```

quelles conclusions tirez-vous? 
- au sujet du volume de donnée? 
- de ce qu'il conviendrait de faire?

In [10]:
## on génère le dictionnaire via reduce
word_count = reduce(cleaning_map("livre01.txt"))


# import de la bibliothèque
import seaborn as sns 
import matplotlib.pyplot as plt

#modifier la taille de la figure pour l'adapter
sns.set(rc={'figure.figsize':(200,100)})

# afficher un histogramme. les données doivent être dans une liste
# transformez donc vos clés et valeurs du dictionnaire en liste
# idéalement ordonnez vos clés

word_count = sorted(word_count.items(), key= lambda item : item[1], reverse=True)
words = [word for word,count in word_count]
counts = [count for word,count in word_count]


sns.barplot(data=None,x = words, y = counts)

fort --> 2
fourmi --> 2
dit --> 2
est --> 2
cigale --> 1
ayant --> 1
chanté --> 1
été --> 1
trouva --> 1
dépourvue --> 1
bise --> 1
fut --> 1
venue --> 1
seul --> 1
petit --> 1
morceau --> 1
mouche --> 1
vermisseau --> 1
alla --> 1
crier --> 1
famine --> 1
voisine --> 1
priant --> 1
prêter --> 1
grain --> 1
subsister --> 1
jusqu --> 1
saison --> 1
nouvelle --> 1
paierai --> 1
avant --> 1
août --> 1
foi --> 1
animal --> 1
intérêt --> 1
principal --> 1
prêteuse --> 1
moindre --> 1
défaut --> 1
faisiez --> 1
temps --> 1
chaud --> 1
emprunteuse --> 1
nuit --> 1
jour --> 1
venant --> 1
chantais --> 1
déplaise --> 1
chantiez --> 1
suis --> 1
aise --> 1
bien --> 1
dansez --> 1
maintenant --> 1


<AxesSubplot: >

<u><b>On remarque:</b></u>
- plus le volume de donnée est important, plus la lisibilité du graphique est moindre.
- Seul quelques mots ont des occurences vraiment élevés, notamment les formes conjuguées des verbes être et avoir, de certaines prépositions etc...

<u><b>Il faudrait:</b></u>

- Rendre le comptage des mots <u><b>"case insensitive"</b></u> en mettant tout en minuscule pour éviter de nouvelles colonnes superflues.
- Songer à supprimer de notre graphique les mots dont les occurences sont assez faibles en tronquant les listes words et counts à partir d'une certaine valeur 
- Il faudrait aussi ajouter au fichier stop_words_fr.txt la liste de tous les pronoms,articles,préposition,conjonctions.

- Il faudrait aussi affiner le stemming en mettant à jour notre dictionnaire avec beaucoup plus de mots.